# First Model - InceptionNet

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, AveragePooling2D, Concatenate, Flatten, Dense

def inception_block(x, filters):
    conv1x1 = Conv2D(filters[0], (1, 1), padding='same', activation='relu')(x)

    conv3x3 = Conv2D(filters[1], (1, 1), padding='same', activation='relu')(x)
    conv3x3 = Conv2D(filters[2], (3, 3), padding='same', activation='relu')(conv3x3)

    conv5x5 = Conv2D(filters[3], (1, 1), padding='same', activation='relu')(x)
    conv5x5 = Conv2D(filters[4], (5, 5), padding='same', activation='relu')(conv5x5)

    maxpool = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    maxpool = Conv2D(filters[5], (1, 1), padding='same', activation='relu')(maxpool)

    inception_block = Concatenate(axis=-1)([conv1x1, conv3x3, conv5x5, maxpool])
    return inception_block

def inception_net(input_shape, num_classes):
    input_tensor = Input(shape=input_shape)

    # Initial convolutional layer
    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same', activation='relu')(input_tensor)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    # Inception blocks
    x = inception_block(x, filters=[64, 128, 128, 32, 32, 32])
    x = inception_block(x, filters=[128, 192, 192, 64, 64, 64])

    # MaxPooling and Fully Connected layers
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dense(num_classes, activation='softmax')(x)

    model = tf.keras.models.Model(inputs=input_tensor, outputs=x)

    return model

# Set input shape and number of classes
input_shape = (224, 224, 3)  # Example input shape, adjust based on your requirements
num_classes = 10  # Adjust based on your classification task

# Create the InceptionNet model
model = inception_net(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()


# Second Model - ResNet

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, Add, GlobalAveragePooling2D, Dense

def residual_block(x, filters, kernel_size=3, strides=1):
    # Shortcut connection
    shortcut = x

    # First convolutional layer
    x = Conv2D(filters, kernel_size=kernel_size, strides=strides, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # Second convolutional layer
    x = Conv2D(filters, kernel_size=kernel_size, strides=1, padding='same')(x)
    x = BatchNormalization()(x)

    # Shortcut connection
    if strides != 1 or shortcut.shape[-1] != filters:
        shortcut = Conv2D(filters, kernel_size=1, strides=strides, padding='same')(shortcut)
        shortcut = BatchNormalization()(shortcut)

    # Element-wise addition
    x = Add()([x, shortcut])
    x = ReLU()(x)

    return x

def resnet(input_shape, num_classes, num_blocks=[2, 2, 2, 2]):
    input_tensor = Input(shape=input_shape)

    # Initial convolutional layer
    x = Conv2D(64, (7, 7), strides=2, padding='same')(input_tensor)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # Residual blocks
    for block_num in range(len(num_blocks)):
        filters = 64 * (2 ** block_num)
        for _ in range(num_blocks[block_num]):
            if block_num > 0 and _ == 0:
                # Downsample for the first block in each group (except the first group)
                x = residual_block(x, filters, strides=2)
            else:
                x = residual_block(x, filters)

    # Global average pooling and dense layer
    x = GlobalAveragePooling2D()(x)
    output_tensor = Dense(num_classes, activation='softmax')(x)

    model = tf.keras.models.Model(inputs=input_tensor, outputs=output_tensor)

    return model

# Set input shape and number of classes
input_shape = (224, 224, 3)  # Example input shape, adjust based on your requirements
num_classes = 10  # Adjust based on your classification task

# Create the ResNet model
model = resnet(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()


# Third Model - InceptionNetResNet

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, Concatenate, AveragePooling2D, GlobalAveragePooling2D, Dense

def inception_resnet_block(x, scale=0.1):
    conv1 = Conv2D(32, (1, 1), padding='same', activation='relu')(x)

    conv2_1 = Conv2D(32, (1, 1), padding='same', activation='relu')(x)
    conv2_1 = Conv2D(32, (3, 3), padding='same', activation='relu')(conv2_1)

    conv3_1 = Conv2D(32, (1, 1), padding='same', activation='relu')(x)
    conv3_1 = Conv2D(32, (5, 5), padding='same', activation='relu')(conv3_1)

    inception_block = Concatenate(axis=-1)([conv1, conv2_1, conv3_1])

    # Residual connection
    scaled_block = Conv2D(int(x.shape[-1]), (1, 1), padding='same', activation='linear')(inception_block)
    scaled_block = BatchNormalization()(scaled_block)

    out = Concatenate(axis=-1)([x, scaled_block])
    out = ReLU()(out * scale)

    return out

def inception_resnet(input_shape, num_classes):
    input_tensor = Input(shape=input_shape)

    # Initial convolutional layer
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(input_tensor)

    # Inception-ResNet blocks
    for _ in range(5):  # You can adjust the number of blocks as needed
        x = inception_resnet_block(x)

    # Global average pooling and dense layer
    x = GlobalAveragePooling2D()(x)
    output_tensor = Dense(num_classes, activation='softmax')(x)

    model = tf.keras.models.Model(inputs=input_tensor, outputs=output_tensor)

    return model

# Set input shape and number of classes
input_shape = (224, 224, 3)  # Example input shape, adjust based on your requirements
num_classes = 10  # Adjust based on your classification task

# Create the Inception-ResNet model
model = inception_resnet(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()


# Fourth Model - DepthwiseInceptionNet

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Concatenate, Conv2D, MaxPooling2D, AveragePooling2D, DepthwiseConv2D, BatchNormalization, ReLU, GlobalAveragePooling2D, Dense

def depthwise_inception_net(input_shape, num_classes):
    input_tensor = Input(shape=input_shape)

    # Inception module 1
    tower_1 = Conv2D(64, (1, 1), padding='same', activation='relu')(input_tensor)

    tower_2 = Conv2D(64, (1, 1), padding='same', activation='relu')(input_tensor)
    tower_2 = DepthwiseConv2D(3, padding='same', activation='relu')(tower_2)

    tower_3 = Conv2D(64, (1, 1), padding='same', activation='relu')(input_tensor)
    tower_3 = DepthwiseConv2D(5, padding='same', activation='relu')(tower_3)

    tower_4 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(input_tensor)
    tower_4 = Conv2D(64, (1, 1), padding='same', activation='relu')(tower_4)

    inception_block_1 = Concatenate(axis=-1)([tower_1, tower_2, tower_3, tower_4])

    # Inception module 2
    tower_5 = Conv2D(128, (1, 1), padding='same', activation='relu')(inception_block_1)

    tower_6 = Conv2D(128, (1, 1), padding='same', activation='relu')(inception_block_1)
    tower_6 = DepthwiseConv2D(3, padding='same', activation='relu')(tower_6)

    tower_7 = Conv2D(128, (1, 1), padding='same', activation='relu')(inception_block_1)
    tower_7 = DepthwiseConv2D(5, padding='same', activation='relu')(tower_7)

    tower_8 = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(inception_block_1)
    tower_8 = Conv2D(128, (1, 1), padding='same', activation='relu')(tower_8)

    inception_block_2 = Concatenate(axis=-1)([tower_5, tower_6, tower_7, tower_8])

    # Global average pooling and dense layers
    x = GlobalAveragePooling2D()(inception_block_2)
    output_tensor = Dense(num_classes, activation='softmax')(x)

    model = tf.keras.models.Model(inputs=input_tensor, outputs=output_tensor)

    return model

# Set input shape and number of classes
input_shape = (224, 224, 3)  # Example input shape, adjust based on your requirements
num_classes = 10  # Adjust based on your classification task

# Create the depthwise separable InceptionNet-style CNN model
model = depthwise_inception_net(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()


# Fifth Model - DepthwiseCNN

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, DepthwiseConv2D, BatchNormalization, ReLU, GlobalAveragePooling2D, Dense

# Define the depthwise separable CNN model
def depthwise_resnet(input_shape, num_classes):
    model = Sequential()

    # Depthwise separable convolution block 1
    model.add(DepthwiseConv2D(3, padding='same', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(ReLU())
    model.add(Conv2D(64, (1, 1)))
    model.add(BatchNormalization())
    model.add(ReLU())

    # Depthwise separable convolution block 2
    model.add(DepthwiseConv2D(3, strides=(2, 2), padding='same'))
    model.add(BatchNormalization())
    model.add(ReLU())
    model.add(Conv2D(128, (1, 1)))
    model.add(BatchNormalization())
    model.add(ReLU())

    # Depthwise separable convolution block 3
    model.add(DepthwiseConv2D(3, strides=(2, 2), padding='same'))
    model.add(BatchNormalization())
    model.add(ReLU())
    model.add(Conv2D(256, (1, 1)))
    model.add(BatchNormalization())
    model.add(ReLU())

    # Global average pooling and dense layers
    model.add(GlobalAveragePooling2D())
    model.add(Dense(num_classes, activation='softmax'))

    return model

# Set input shape and number of classes
input_shape = (224, 224, 3)  # Example input shape, adjust based on your requirements
num_classes = 10  # Adjust based on your classification task

# Create the depthwise separable CNN model
model = depthwise_resnet(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()


# Seventh Model - DepthwiseResNet

In [ ]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the Depthwise Separable Residual Block
def depthwise_res_block(x, filters, kernel_size=3, strides=1):
    # Shortcut connection
    shortcut = x
    
    # Depthwise separable convolution
    x = layers.DepthwiseConv2D(kernel_size, strides=strides, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    # Pointwise convolution
    x = layers.Conv2D(filters, kernel_size=1, strides=1, padding='same')(x)
    x = layers.BatchNormalization()(x)
    
    # Add the shortcut to the main path
    x = layers.add([x, shortcut])
    x = layers.Activation('relu')(x)
    
    return x

# Define the Depthwise Separable ResNet CNN model
def depthwise_resnet(input_shape, num_classes):
    model = models.Sequential()
    
    # Initial convolution block
    model.add(layers.Conv2D(64, (7, 7), strides=(2, 2), input_shape=input_shape, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
    
    # Depthwise Residual Blocks
    model.add(depthwise_res_block(model.output, 64, strides=1))
    model.add(depthwise_res_block(model.output, 64, strides=1))
    model.add(depthwise_res_block(model.output, 128, strides=2))
    model.add(depthwise_res_block(model.output, 128, strides=1))
    model.add(depthwise_res_block(model.output, 256, strides=2))
    model.add(depthwise_res_block(model.output, 256, strides=1))
    model.add(layers.GlobalAveragePooling2D())
    
    # Fully connected layer
    model.add(layers.Dense(num_classes, activation='softmax'))
    
    return model

# Specify input shape and number of classes
input_shape = (224, 224, 3)  # Adjust the input shape based on your dataset
num_classes = 10  # Adjust the number of classes based on your problem

# Create the model
model = depthwise_resnet(input_shape, num_classes)

# Compile the model


# Seventh Model - DepthwiseInceptionResNet

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Concatenate, Conv2D, MaxPooling2D, AveragePooling2D, DepthwiseConv2D, BatchNormalization, ReLU, GlobalAveragePooling2D, Dense, Add

def depthwise_inception_resnet(input_shape, num_classes):
    input_tensor = Input(shape=input_shape)

    # Inception-ResNet module
    def inception_resnet_block(x, scale=1.0):
        conv1 = Conv2D(32, (1, 1), padding='same', activation='relu')(x)

        conv2_1 = Conv2D(32, (1, 1), padding='same', activation='relu')(x)
        conv2_1 = DepthwiseConv2D(3, padding='same', activation='relu')(conv2_1)

        conv2_2 = Conv2D(32, (1, 1), padding='same', activation='relu')(x)
        conv2_2 = DepthwiseConv2D(5, padding='same', activation='relu')(conv2_2)

        inception_block = Concatenate(axis=-1)([conv1, conv2_1, conv2_2])

        # Residual connection
        scaled_block = Conv2D(int(x.shape[-1]), (1, 1), padding='same', activation='linear')(inception_block)
        scaled_block = BatchNormalization()(scaled_block)

        out = Add()([x, scaled_block])
        out = ReLU()(out * scale)

        return out

    # Initial convolution
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(input_tensor)

    # Depthwise Inception-ResNet blocks
    x = inception_resnet_block(x)
    x = inception_resnet_block(x)
    x = inception_resnet_block(x)

    # Global average pooling and dense layers
    x = GlobalAveragePooling2D()(x)
    output_tensor = Dense(num_classes, activation='softmax')(x)

    model = tf.keras.models.Model(inputs=input_tensor, outputs=output_tensor)

    return model

# Set input shape and number of classes
input_shape = (224, 224, 3)  # Example input shape, adjust based on your requirements
num_classes = 10  # Adjust based on your classification task

# Create the depthwise Inception-ResNet-style CNN model
model = depthwise_inception_resnet(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()


# Eighth Model - Depthwise Separable CNN

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the Depthwise Separable CNN model
def depthwise_separable_cnn(input_shape, num_classes):
    model = models.Sequential()
    
    # Depthwise separable convolution block
    model.add(layers.DepthwiseConv2D(3, strides=(1, 1), input_shape=input_shape, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.Conv2D(64, (1, 1), strides=(1, 1), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.GlobalAveragePooling2D())
    
    # Fully connected layer
    model.add(layers.Dense(num_classes, activation='softmax'))
    
    return model

# Specify input shape and number of classes
input_shape = (224, 224, 3)  # Adjust the input shape based on your dataset
num_classes = 10  # Adjust the number of classes based on your problem

# Create the model
depthwise_separable_model = depthwise_separable_cnn(input_shape, num_classes)

# Compile the model
depthwise_separable_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
depthwise_separable_model.summary()


# Seventh Model - Separable CNN

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the Separable CNN model
def separable_cnn(input_shape, num_classes):
    model = models.Sequential()
    
    # Separable convolution block
    model.add(layers.SeparableConv2D(64, (3, 3), strides=(1, 1), input_shape=input_shape, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.GlobalAveragePooling2D())
    
    # Fully connected layer
    model.add(layers.Dense(num_classes, activation='softmax'))
    
    return model

# Specify input shape and number of classes
input_shape = (224, 224, 3)  # Adjust the input shape based on your dataset
num_classes = 10  # Adjust the number of classes based on your problem

# Create the model
separable_model = separable_cnn(input_shape, num_classes)

# Compile the model
separable_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
separable_model.summary()
